# Set-Up

In [1]:
!pip install pyvista
!pip install matplotlib
!pip install pygmsh
!pip install panel

     |████████████████████████████████| 1.4 MB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 59.5 MB 32 kB/s  eta 0:00:014    |█▍                              | 2.6 MB 12.6 MB/s eta 0:00:05     |██                              | 3.7 MB 12.6 MB/s eta 0:00:05     |███████▌                        | 13.9 MB 8.4 MB/s eta 0:00:06     |████████                        | 14.9 MB 8.4 MB/s eta 0:00:06     |███████████▊                    | 21.8 MB 10.0 MB/s eta 0:00:04     |███████████████████             | 35.4 MB 8.4 MB/s eta 0:00:03     |███████████████████▉            | 36.8 MB 8.4 MB/s eta 0:00:03     |██████████████████████▏         | 41.2 MB 8.6 MB/s eta 0:00:03     |████████████████████████▌       | 45.5 MB 8.6 MB/s eta 0:00:02     |██████████████████████████▍     | 49.0 MB 7.3 MB/s eta 0:00:02     |███████████████████████████     | 50.1 

In [2]:
# MUST RUN THIS CELL:
!pip uninstall -y h5py
!pip install h5py==2.9.0

     |████████████████████████████████| 287 kB 8.0 kB/s eta 0:00:01
  Created wheel for h5py: filename=h5py-2.9.0-cp39-cp39-linux_x86_64.whl size=4691340 sha256=ea5f50296b97fc508383b52a858b31e8a79456f401031ad78a72d108c467dae9
  Stored in directory: /root/.cache/pip/wheels/ef/54/5c/3fbdb9cfe071661699815cfd6b71ddf2d12d61d121a109a5e0
Successfully built h5py


In [74]:
import dolfinx
import numpy as np
from mpi4py import MPI
from dolfinx.cpp.mesh import CellType
import dolfinx.io
import pyvista
import dolfinx.plot
from math import sin, cos, pi, ceil, floor
import pygmsh
import gmsh
import meshio
import ufl
from petsc4py import PETSc
import os

# Function Definitions

In [72]:
def mesh_from_obj(obj_dir):
    
    if not os.path.isdir('breast.xdmf'):
        breast_mesh = meshio.read(obj_dir)
        meshio.write('breast.xdmf', breast_mesh)
        
    with dolfinx.io.XDMFFile(MPI.COMM_WORLD, "breast.xdmf", 'r') as f:
        mesh = f.read_mesh(name="Grid")
    
    return mesh

## Function to Apply Load to Breast

In [5]:
def apply_loading(obj_dir, E, beam_angle, nu, rho, g, elem_order, num_steps):
    
    beam_angle *= pi/180
    
    mesh = mesh_from_obj(obj_dir)
    V = dolfinx.VectorFunctionSpace(mesh, ("CG", elem_order))
    
    # Create lambda and mu fields:
    lambda_ = E*nu/((1+nu)*(1-2*nu))
    mu = E/(2*(1+nu))
    
    # Apply fixed BC:
    fixed = lambda x: x[0] < 10
    fixed_facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, fixed)
    facet_tag = dolfinx.MeshTags(mesh, mesh.topology.dim-1, fixed_facets, 1)
    u_bc = dolfinx.Function(V)
    with u_bc.vector.localForm() as loc:
        loc.set(0)
    left_dofs = dolfinx.fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.indices[facet_tag.values==1])
    bcs = [dolfinx.DirichletBC(u_bc, left_dofs)]
    
    B = dolfinx.Constant(mesh, (0, 0, 0))
    T = dolfinx.Constant(mesh, (0, 0, 0))
    v = ufl.TestFunction(V)
    u = dolfinx.Function(V)

    d = len(u)
    I = ufl.variable(ufl.Identity(d))
    F = ufl.variable(I + ufl.grad(u))
    C = ufl.variable(F.T * F)
    Ic = ufl.variable(ufl.tr(C))
    J  = ufl.variable(ufl.det(F))
    psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lambda_ / 2) * (ufl.ln(J))**2
    P = ufl.diff(psi, F)
    
    metadata = {"quadrature_degree": elem_order}
    ds = ufl.Measure('ds', subdomain_data=facet_tag, metadata=metadata)
    dx = ufl.Measure("dx", metadata=metadata)
    F = ufl.inner(ufl.grad(v), P)*dx - ufl.inner(v, B)*dx - ufl.inner(v, T)*ds(2) 
    
    problem = dolfinx.fem.NonlinearProblem(F, u, bcs)
    solver = dolfinx.NewtonSolver(MPI.COMM_WORLD, problem)

    solver.atol = 1e-3
    solver.rtol = 1e-3
    solver.convergence_criterion = "incremental"
    
    f_step = rho*g/num_steps
    for n in range(num_steps):
        print(f"Performing load step {n+1}/{num_steps}")
        B.value[0] = (n+1)*f_step*cos(beam_angle)
        B.value[2] = -(n+1)*f_step*sin(beam_angle)
        num_its, converged = solver.solve(u)
        assert(converged)
        u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    
    return (u, mesh)

## Function to Plot Beam Deformation

In [82]:
def plot_deformation(mesh, uh):
    
    pyvista.start_xvfb(wait=0.05)
    topology, cell_types = dolfinx.plot.create_vtk_topology(mesh, mesh.topology.dim)
    grid = pyvista.UnstructuredGrid(topology, cell_types, mesh.geometry.x)
    
    p = pyvista.Plotter(window_size=[960,480]) #
    
    p.add_text("Deformed configuration", name="title", position="upper_edge")
    
    grid["u"] = uh.compute_point_values().real 
    actor_0 = p.add_mesh(grid, style="wireframe", color="k")
    warped = grid.warp_by_vector("u", factor=1.5)
    actor_1 = p.add_mesh(warped)
    
    p.show_axes()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

# Call Functions

In [93]:
# Fixed parameters:
elem_order = 2
W = 40 # in mm
L = 90 # in mm
nu = 0.33 # dimensionless
rho = 0.00102 # in g mm^-3
g = 9.81 # in m s^-2
num_steps = 5

# Variables:
beam_angle = 90
E =  5 # in mPa

In [94]:
obj_dir = 'b+hollowUV2_transformed.obj'
uh, mesh = apply_loading(obj_dir, E, beam_angle, nu, rho, g, elem_order, num_steps)

Performing load step 1/5
Performing load step 2/5
Performing load step 3/5
Performing load step 4/5
Performing load step 5/5


In [95]:
plot_deformation(mesh, uh)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, color_mappers=[LinearColorMapper(id='108...], height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)